In [33]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

In [34]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [6]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [ ]:
import json
BASE_MODEL = model_name
question = "what are the sales in augest 2023"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = "auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt=[
  {
    "role": "system",
    "content": "Assume you are a system that detects the date in the given secentance, that date could in any format."},
    {"role":"system",
     "content":"""If a user gives "top 10 orders in march 2020", you should return march 2020"""},
    {"role":"user",
    "content":question}
]
prompt = json.dumps(prompt)
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
model.eval()
response = tokenizer.decode(model.generate(**model_input, max_length=4096)[0], skip_special_tokens=True)
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]